In [12]:
import os
import tensorflow as tf
from tensorflow import keras
# import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

IMAGE_SETS_PATH = 'F:/VOC2012/ImageSets/'
IMAGE_SETS_MAIN_PATH = 'F:/VOC2012/ImageSets/Main/'
JPEG_IMAGES_PATH = 'F:/VOC2012/JPEGImages/'

In [2]:
imageSets_list = os.listdir(IMAGE_SETS_PATH + 'Main')

print(imageSets_list)

['aeroplane_train.txt', 'aeroplane_trainval.txt', 'aeroplane_val.txt', 'bicycle_train.txt', 'bicycle_trainval.txt', 'bicycle_val.txt', 'bird_train.txt', 'bird_trainval.txt', 'bird_val.txt', 'boat_train.txt', 'boat_trainval.txt', 'boat_val.txt', 'bottle_train.txt', 'bottle_trainval.txt', 'bottle_val.txt', 'bus_train.txt', 'bus_trainval.txt', 'bus_val.txt', 'car_train.txt', 'car_trainval.txt', 'car_val.txt', 'cat_train.txt', 'cat_trainval.txt', 'cat_val.txt', 'chair_train.txt', 'chair_trainval.txt', 'chair_val.txt', 'cow_train.txt', 'cow_trainval.txt', 'cow_val.txt', 'diningtable_train.txt', 'diningtable_trainval.txt', 'diningtable_val.txt', 'dog_train.txt', 'dog_trainval.txt', 'dog_val.txt', 'horse_train.txt', 'horse_trainval.txt', 'horse_val.txt', 'motorbike_train.txt', 'motorbike_trainval.txt', 'motorbike_val.txt', 'person_train.txt', 'person_trainval.txt', 'person_val.txt', 'pottedplant_train.txt', 'pottedplant_trainval.txt', 'pottedplant_val.txt', 'sheep_train.txt', 'sheep_trainval.

In [5]:
train_file_list = []
val_file_list = []

for file_name in imageSets_list:
    if file_name.find('_train.txt') != -1:
        train_file_list.append(file_name)
    elif file_name.find('_val.txt') != -1:
        val_file_list.append(file_name)

print(train_file_list)
print(val_file_list)

['aeroplane_train.txt', 'bicycle_train.txt', 'bird_train.txt', 'boat_train.txt', 'bottle_train.txt', 'bus_train.txt', 'car_train.txt', 'cat_train.txt', 'chair_train.txt', 'cow_train.txt', 'diningtable_train.txt', 'dog_train.txt', 'horse_train.txt', 'motorbike_train.txt', 'person_train.txt', 'pottedplant_train.txt', 'sheep_train.txt', 'sofa_train.txt', 'train_train.txt', 'tvmonitor_train.txt']
['aeroplane_val.txt', 'bicycle_val.txt', 'bird_val.txt', 'boat_val.txt', 'bottle_val.txt', 'bus_val.txt', 'car_val.txt', 'cat_val.txt', 'chair_val.txt', 'cow_val.txt', 'diningtable_val.txt', 'dog_val.txt', 'horse_val.txt', 'motorbike_val.txt', 'person_val.txt', 'pottedplant_val.txt', 'sheep_val.txt', 'sofa_val.txt', 'train_val.txt', 'tvmonitor_val.txt']


Generate the train and val dataset

In [31]:
train_dataframe = pd.DataFrame(columns=('file_name', 'class'))
val_dataframe = pd.DataFrame(columns=('file_name', 'class'))

kind = 0
for train_file_name in train_file_list:
    f = open(IMAGE_SETS_MAIN_PATH + train_file_name)
    row_list = f.readlines()
    for pair in row_list:
        target = pair.split()
        if target[1] == '1':
            train_dataframe = train_dataframe.append(
                [{'file_name': JPEG_IMAGES_PATH + target[0] + '.jpg',
                  'class': train_file_name[:-10]}],
                ignore_index=True
            )
    kind += 1
    f.close()

kind = 0
for val_file_name in val_file_list:
    f = open(IMAGE_SETS_MAIN_PATH + val_file_name)
    row_list = f.readlines()
    for pair in row_list:
        target = pair.split()
        if target[1] == '1':
            val_dataframe = val_dataframe.append(
                [{'file_name': JPEG_IMAGES_PATH + target[0] + '.jpg',
                  'class': val_file_name[:-8]}],
                ignore_index=True
            )
    kind += 1
    f.close()


In [37]:
train_generator = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=keras.applications.mobilenet.preprocess_input,
    validation_split=0.1
)

test_generator = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=keras.applications.mobilenet.preprocess_input
)

train_sets = train_generator.flow_from_dataframe(
    dataframe=train_dataframe,
    x_col='file_name',
    y_col='class',
    target_size=(300, 300),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    subset='training',
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_sets = train_generator.flow_from_dataframe(
    dataframe=val_dataframe,
    x_col='file_name',
    y_col='class',
    target_size=(300, 300),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False,
    seed=0,
    subset='validation'
    # rotation_range=30,
    # zoom_range=0.15,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.15,
    # horizontal_flip=True,
    # fill_mode="nearest"
)

Found 7498 validated image filenames belonging to 20 classes.
Found 835 validated image filenames belonging to 20 classes.


Build the MobileNet model.

In [34]:
original_model = keras.applications.mobilenet.MobileNet(
    input_shape=(300, 300, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

inputs = original_model.input

x = keras.layers.Dense(128, activation='relu')(original_model.output)
x = keras.layers.Dense(128, activation='relu')(x)

outputs = keras.layers.Dense(20, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

17235968/17225924 [==============================] - 2s 0us/step


In [35]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300, 300, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 150, 150, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 150, 150, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 150, 150, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 150, 150, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 150, 150, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 150, 150, 32)      0     

Train the model using VOC2012.

In [38]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        './checkpoints/weights.{epoch:02d}-{val_loss:.2f}-{val_accuracy:.4f}.h5',
        verbose=1,
        save_weights_only=True
    )
]

In [39]:
model.fit(
    train_sets,
    validation_data=val_sets,
    epochs=5,
    verbose=1,
    callbacks=callbacks
)

Epoch 1/5


ResourceExhaustedError:  OOM when allocating tensor with shape[512,512,1,1] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/conv_pw_7/Conv2D (defined at <ipython-input-39-2d3c675e4de2>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_5579]

Function call stack:
train_function


In [ ]:
model.valid
